# 猫狗大战（Dogs vs Cats）

## 项目介绍

使用要迁移学习的模型，将数据集中的bottleneck特征提取出来，方便后续训练新模型。
 

## 1. 导入数据集

读取datas目录下的数据集，并分为train和test两类。

In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob

data = load_files('datas')

files = np.array(data['filenames'])
targets = np.array(data['target'])
target_names = np.array(data['target_names'])

train_files = [files[idx] for idx in range(len(files)) if targets[idx] == np.argwhere(target_names == 'train')]
test_files = [files[idx] for idx in range(len(files)) if targets[idx] == np.argwhere(target_names == 'test')]

print("There are {} train images.".format(len(train_files)))
print("There are {} test images.".format(len(test_files)))

Using TensorFlow backend.


There are 25000 train images.
There are 12500 test images.


<a id="step1"></a>
## 2. 数据分析

### 2.1 提取数据特征

通过查看训练数据集，发现数据集的标注信息是定义在文件名的，从文件名中提取对应的数据特征，如下表：

|文件|标注（dog）|标注（cat）|
|-|-|-|
|datas/train/cat.6938.jpg  |0|1|
|datas/train/dog.11432.jpg |1|0|
|datas/train/cat.433.jpg   |0|1|
|datas/train/cat.11305.jpg |0|1|


In [2]:
data_labels = ("dog", "cat")

train_labels = []
for file in train_files:
    for idx in range(len(data_labels)):
        if data_labels[idx] in file:
            train_labels.append(idx)

train_targets = np_utils.to_categorical(np.array(train_labels), 2)
print("The first 5 train file:\n{}\n".format(train_files[0:5]))
print("The first 5 train targets:\n{}\n".format(train_targets[0:5]))

The first 5 train file:
['datas/train/cat.9436.jpg', 'datas/train/cat.6938.jpg', 'datas/train/dog.11432.jpg', 'datas/train/cat.10939.jpg', 'datas/train/cat.433.jpg']

The first 5 train targets:
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]



## 2.2 数据处理

根据文件名，读取图片文件中的数据

In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # 用PIL加载RGB图像为PIL.Image.Image类型
    img = image.load_img(img_path, target_size=(224, 224))
    # 将PIL.Image.Image类型转化为格式为(224, 224, 3)的3维张量
    x = image.img_to_array(img)
    # 将3维张量转化为格式为(1, 224, 224, 3)的4维张量并返回
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# 加载图片资源
train_tensors = paths_to_tensor(train_files)
test_tensors = paths_to_tensor(test_files)

100%|██████████| 12500/12500 [00:35<00:00, 352.93it/s]


### 2.2 验证集划分

从sklearn.model_selection中导入train_test_split
将train_files和train_targets作为train_test_split的输入变量
设置test_size为0.2，划分出20%的验证集，80%的数据留作新的训练集。
设置random_state随机种子，以确保每一次运行都可以得到相同划分的结果。（随机种子固定，生成的随机序列就是确定的）

In [5]:
from sklearn.model_selection import train_test_split

X_train , X_valid , y_train, y_valid = train_test_split(train_tensors, train_targets, test_size=0.2, random_state=100)

print("Splited train set num: {}".format(len(X_train)))
print("Splited valid set num: {}".format(len(X_valid)))

Splited train set num: 20000
Splited valid set num: 5000


<a id="step2"></a>
## 3. 迁移学习

基于[Deep Residual Networks](https://github.com/KaimingHe/deep-residual-networks#models) 来创建CNN模型。

**TODO: 介绍为什么使用ResNet-50，以及ResNet模型的原理**

### 3.1 迁移无全连接层的模型

为了提高模型的训练速度和质量，这里使用迁移学习。

In [6]:
from keras.applications.resnet50 import ResNet50

# 使用经过imagenet训练的无全连接层ResNet50模型
resnet_model = ResNet50(include_top=False, weights='imagenet')
resnet_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256                                          
___________________________________________________________________________________________

### 3.2 提取数据集的特征数据

使用迁移的模型分别提取无全连接层的训练集、验证集和测试集的数据特征。

In [7]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

# 提取训练集特征
train_input = preprocess_input(X_train)
train_feature = resnet_model.predict(train_input, verbose=1)
print(train_feature.shape)

# 提取验证集特征
valid_input = preprocess_input(X_valid)
valid_feature = resnet_model.predict(valid_input, verbose=1)
print(valid_feature.shape)

# 提取测试集特征
test_input = preprocess_input(test_tensors)
test_feature = resnet_model.predict(test_input, verbose=1)
print(test_feature.shape)

20000/20000 [==============================] - 245s   
(20000, 1, 1, 2048)
5000/5000 [==============================] - 61s    
(5000, 1, 1, 2048)
12500/12500 [==============================] - 153s   
(12500, 1, 1, 2048)


### 3.3 保存特征数据

将上述提取的特征数据保存到文件中。

In [8]:
# 将特征数据保存到bottlenecks
np.savez('bottlenecks/bottlenecks_resnet50.npz', train=train_feature, valid=valid_feature, test=test_feature)